In [ ]:
import pandas as pd
import requests
import time
import os
import re

In [ ]:
items_info_cleaned_2_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat', sep='\t', on_bad_lines='skip', index_col=False,)
items_info_cleaned_2_df.head()

,Book_ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
1,2,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
2,3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
3,4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...
4,5,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...


In [ ]:
import pandas as pd
import requests
import time
import os
import re

# --- CONFIGURATION ---
API_KEY = "59541_7b280637020d60a99abb89a246a1df18"
BASE_URL = "https://api2.isbndb.com/book/"
SAVE_FILE = "/content/isbn_enriched_data.csv"

headers = {"Authorization": API_KEY}

# --- Load dataset ---
url = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat'
items_info_cleaned_2_df = pd.read_csv(url, sep='\t', on_bad_lines='skip', index_col=False)

# Extract ISBNs
isbn_list = items_info_cleaned_2_df['ISBN'].astype(str).unique().tolist()

# --- Function to validate ISBNs ---
def is_valid_isbn(isbn):
    """Checks if an ISBN is valid (either 10 or 13 digits, allowing 'X' at the end for ISBN-10)."""
    return bool(re.match(r'^\d{9}[\dX]$|^\d{13}$', isbn))

# --- Function to fetch book data ---
def fetch_book_data(isbn):
    try:
        url = f"{BASE_URL}{isbn}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json().get('book', {})
            return {
                'isbn10': data.get('isbn10', ''),
                'isbn13': data.get('isbn13', ''),
                'subjects': "|".join(data.get('subjects', [])),  # Convert list to string
                'synopsis': data.get('synopsis', ''),  # May be missing for some books
                'language': data.get('language', ''),
                'pages': data.get('pages', '')
            }
        else:
            return None
    except Exception as e:
        print(f"Error fetching {isbn}: {e}")
        return None

# --- TEST: Fetch First 5 ISBNs Only ---
test_isbn_list = isbn_list[:5]  # Select only the first 5 ISBNs
test_valid_isbns = [isbn for isbn in test_isbn_list if is_valid_isbn(isbn)]

print(f"Testing with {len(test_valid_isbns)} valid ISBNs...")

# --- Run API calls for the first 5 valid ISBNs ---
test_results = []
for isbn in test_valid_isbns:
    book_info = fetch_book_data(isbn)
    if book_info:
        test_results.append(book_info)
    time.sleep(1)  # API rate limit

# Convert test results to a DataFrame and display
test_results_df = pd.DataFrame(test_results)
test_results_df.head()

Testing with 5 valid ISBNs...


,isbn10,isbn13,subjects,synopsis,language,pages
0,0060973129,9780060973124,History|Military|World War II,An outstanding military history that offers a ...,en,555
1,0393045218,9780393045215,Arts & Photography|Decorative Arts & Design|Te...,"Barber, one the world's leading authorities on...",en,240
2,0425176428,9780425176429,History|Historical Study & Educational Resourc...,With its in-depth reflections on the monumenta...,en,416
3,0452264464,9780452264465,Literature & Fiction|Genre Fiction|Historical|...,"Sethe, an escaped slave living in post-Civil W...",en,288
4,0609804618,9780609804612,Humor & Entertainment|Humor|Parodies|Political...,The Onion has quickly become the world's most ...,en,176


In [ ]:
import pandas as pd
import requests
import time
import os
import re

# --- CONFIGURATION ---
API_KEY = "59541_7b280637020d60a99abb89a246a1df18"
BASE_URL = "https://api2.isbndb.com/book/"
BATCH_SIZE = 10  # Since the API allows max 10 results per call
DAILY_LIMIT = 2000  # Academic plan limit
SAVE_FILE = "/content/isbn_enriched_data.csv"  # Where enriched data is saved
SKIPPED_FILE = "/content/skipped_isbns.csv"  # Log of invalid ISBNs

headers = {"Authorization": API_KEY}

# --- Load dataset ---
url = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat'
items_info_cleaned_2_df = pd.read_csv(url, sep='\t', on_bad_lines='skip', index_col=False)

# Extract unique ISBNs
isbn_list = items_info_cleaned_2_df['ISBN'].astype(str).unique().tolist()

# --- Function to validate ISBNs ---
def is_valid_isbn(isbn):
    """Checks if an ISBN is valid (either 10 or 13 digits, allowing 'X' at the end for ISBN-10)."""
    return bool(re.match(r'^\d{9}[\dX]$|^\d{13}$', isbn))

# Filter valid and invalid ISBNs
valid_isbns = [isbn for isbn in isbn_list if is_valid_isbn(isbn)] # [:10] IF TRYING FOR FIRST 10
invalid_isbns = [isbn for isbn in isbn_list if not is_valid_isbn(isbn)]

# Save invalid ISBNs for reference
pd.DataFrame({'Invalid_ISBNs': invalid_isbns}).to_csv(SKIPPED_FILE, index=False)

print(f"Total valid ISBNs to process: {len(valid_isbns)}")
print(f"{len(invalid_isbns)} invalid ISBNs logged in {SKIPPED_FILE}.")

# --- Resume from saved progress ---
if os.path.exists(SAVE_FILE):
    enriched_df = pd.read_csv(SAVE_FILE)
    processed_isbns = set(enriched_df['isbn10'].astype(str)) | set(enriched_df['isbn13'].astype(str))
    print(f"Resuming from saved progress. {len(processed_isbns)} books already processed.")
else:
    enriched_df = pd.DataFrame(columns=['isbn10', 'isbn13', 'subjects', 'synopsis', 'language', 'pages'])
    processed_isbns = set()

# --- Function to fetch book data ---
def fetch_book_data(isbn):
    try:
        url = f"{BASE_URL}{isbn}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json().get('book', {})
            return {
                'isbn10': data.get('isbn10', ''),
                'isbn13': data.get('isbn13', ''),
                'subjects': "|".join(data.get('subjects', [])),  # Convert list to string
                'synopsis': data.get('synopsis', ''),
                'language': data.get('language', ''),
                'pages': data.get('pages', '')
            }
        else:
            print(f"Error fetching {isbn}: {response.status_code} - {response.json()}")
            return None
    except Exception as e:
        print(f"Exception for {isbn}: {e}")
        return None

# --- Process in batches (Respecting API Limits) ---
total_calls = 0  # Track API calls

for i in range(0, len(valid_isbns), BATCH_SIZE):
    if total_calls >= DAILY_LIMIT:
        print("Daily API limit reached. Stopping for today.")
        break

    batch_isbns = valid_isbns[i:i + BATCH_SIZE]
    batch_results = []

    print(f"Processing batch {i // BATCH_SIZE + 1}/{len(valid_isbns) // BATCH_SIZE + 1}...")

    for isbn in batch_isbns:
        if isbn in processed_isbns:  # Skip already processed ISBNs
            continue

        book_info = fetch_book_data(isbn)
        if book_info:
            batch_results.append(book_info)
            processed_isbns.add(book_info['isbn10'])
            processed_isbns.add(book_info['isbn13'])
            total_calls += 1

        if total_calls >= DAILY_LIMIT:
            print("Daily API limit reached. Stopping for today.")
            break  # Stop processing if limit is reached

        time.sleep(1)  # Prevent hitting API rate limits

    # Convert batch results to DataFrame and append to CSV
    if batch_results:
        batch_df = pd.DataFrame(batch_results)
        enriched_df = pd.concat([enriched_df, batch_df], ignore_index=True)
        enriched_df.to_csv(SAVE_FILE, index=False)  # Save progress

    print(f"Batch {i // BATCH_SIZE + 1} completed. Progress saved.")

print("All valid ISBNs processed successfully or daily limit reached.")

Total valid ISBNs to process: 17360
24 invalid ISBNs logged in /content/skipped_isbns.csv.
Resuming from saved progress. 34716 books already processed.
Processing batch 1/1737...
Batch 1 completed. Progress saved.
Processing batch 2/1737...
Batch 2 completed. Progress saved.
Processing batch 3/1737...
Batch 3 completed. Progress saved.
Processing batch 4/1737...
Batch 4 completed. Progress saved.
Processing batch 5/1737...
Batch 5 completed. Progress saved.
Processing batch 6/1737...
Batch 6 completed. Progress saved.
Processing batch 7/1737...
Batch 7 completed. Progress saved.
Processing batch 8/1737...
Batch 8 completed. Progress saved.
Processing batch 9/1737...
Batch 9 completed. Progress saved.
Processing batch 10/1737...
Batch 10 completed. Progress saved.
Processing batch 11/1737...
Batch 11 completed. Progress saved.
Processing batch 12/1737...
Batch 12 completed. Progress saved.
Processing batch 13/1737...
Batch 13 completed. Progress saved.
Processing batch 14/1737...
Batch 

In [ ]:
# Review Skipped ISBNs
skipped_isbns_df = pd.read_csv("/content/skipped_isbns.csv")
skipped_isbns_df.head()

,Invalid_ISBNs
0,096788330x
1,B00007FYKO
2,042519065x
3,006103181x
4,051513581x


In [ ]:
from google.colab import files

# Download the processed data
files.download("/content/isbn_enriched_data.csv")

# Download skipped ISBNs
files.download("/content/skipped_isbns.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load enriched data
isbn_enriched_df = pd.read_csv(SAVE_FILE)

# Merge using ISBN-10 (since Book-Crossing uses ISBN-10)
merged_df = items_info_cleaned_2_df.merge(isbn_enriched_df, left_on='ISBN', right_on='isbn10', how='left')

# Drop redundant columns
merged_df = merged_df.drop(columns=['isbn10', 'isbn13'])

# Save the final enriched dataset
merged_df.to_csv("/content/Enhanced_Book_Crossing.csv", index=False)

# Download it
from google.colab import files
files.download("/content/Enhanced_Book_Crossing.csv")


This is where I start playing around with things to have a fully detailed dataset:

In [ ]:
import pandas as pd
import requests
import time
import os
import re

# --- Load dataset ---
url1 = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat'
items_info_cleaned_2_df = pd.read_csv(url1, sep='\t', on_bad_lines='skip', index_col=False)


url2 = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/isbn_enriched_data.csv'
items_enriched_data = pd.read_csv(url2, sep=',', on_bad_lines='skip', index_col=False)

In [ ]:
items_info_cleaned_2_df.head()


,Book_ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
1,2,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
2,3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
3,4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...
4,5,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...


In [ ]:
items_enriched_data.head()

,isbn10,isbn13,subjects,synopsis,language,pages
0,0060973129,9780060973124,History|Military|World War II,An outstanding military history that offers a ...,en,555.0
1,0393045218,9780393045215,Arts & Photography|Decorative Arts & Design|Te...,"Barber, one the world's leading authorities on...",en,240.0
2,0425176428,9780425176429,History|Historical Study & Educational Resourc...,With its in-depth reflections on the monumenta...,en,416.0
3,0452264464,9780452264465,Literature & Fiction|Genre Fiction|Historical|...,"Sethe, an escaped slave living in post-Civil W...",en,288.0
4,0609804618,9780609804612,Humor & Entertainment|Humor|Parodies|Political...,The Onion has quickly become the world's most ...,en,176.0


In [ ]:
# Rename ISBN column in one of the datasets
items_info_cleaned_2_df.rename(columns={'ISBN': 'ISBN10'}, inplace=True)
items_enriched_data.rename(columns={'isbn10': 'ISBN10'}, inplace=True)


In [ ]:
# Perform an inner merge so that only books with matching ISBNs in both datasets will be kept (we will only lose 24 books by doing this, see invalid ISBNs datatset on github is needed)
enriched_books_df = pd.merge(items_info_cleaned_2_df, items_enriched_data, on='ISBN10', how='inner')
enriched_books_df.head()

,Book_ID,ISBN10,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,isbn13,subjects,synopsis,language,pages
0,1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,9780060973124,History|Military|World War II,An outstanding military history that offers a ...,en,555.0
1,2,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,9780393045215,Arts & Photography|Decorative Arts & Design|Te...,"Barber, one the world's leading authorities on...",en,240.0
2,3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,9780425176429,History|Historical Study & Educational Resourc...,With its in-depth reflections on the monumenta...,en,416.0
3,4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,9780452264465,Literature & Fiction|Genre Fiction|Historical|...,"Sethe, an escaped slave living in post-Civil W...",en,288.0
4,5,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...,9780609804612,Humor & Entertainment|Humor|Parodies|Political...,The Onion has quickly become the world's most ...,en,176.0


In [ ]:
# Drop the images because I don't think we need this
enriched_books_df.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace=True)
enriched_books_df.head()

,Book_ID,ISBN10,Book-Title,Book-Author,Year-Of-Publication,Publisher,isbn13,subjects,synopsis,language,pages
0,1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,9780060973124,History|Military|World War II,An outstanding military history that offers a ...,en,555.0
1,2,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,9780393045215,Arts & Photography|Decorative Arts & Design|Te...,"Barber, one the world's leading authorities on...",en,240.0
2,3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,9780425176429,History|Historical Study & Educational Resourc...,With its in-depth reflections on the monumenta...,en,416.0
3,4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,9780452264465,Literature & Fiction|Genre Fiction|Historical|...,"Sethe, an escaped slave living in post-Civil W...",en,288.0
4,5,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,9780609804612,Humor & Entertainment|Humor|Parodies|Political...,The Onion has quickly become the world's most ...,en,176.0


In [ ]:
# Check for NaN values in each column
empty_columns = enriched_books_df.isna().sum()
# Print columns with missing values
print(empty_columns[empty_columns > 0])

subjects     56
synopsis    458
pages       868
dtype: int64


In [ ]:
# Identify the rows where the 'subject' column has missing values (NaN)
missing_subject_rows = enriched_books_df[enriched_books_df['subjects'].isna()]

# Print out the columns for those rows
print("Columns with missing subjects:")
print(missing_subject_rows)


Columns with missing subjects:
       Book_ID      ISBN10                                         Book-Title  \
686        687  0894803700                    14,000 Things to Be Happy About   
977        978  0676974562                                          Atonement   
1145      1146  0884092097                                       Kaleidoscope   
1773      1774  9510236756                                Minun sukuni tarina   
1815      1816  0553802550  The Happiest Baby on the Block : The New Way t...   
1932      1933  8448902378                       LÂ®impostor (Antaviana jove)   
2715      2717  1410798860  The Diary of Alice Mary: A Factual Childhood D...   
3168      3170  0679777547                            The Club Dumas: A Novel   
3501      3503  8475060099                                 El momento del ave   
3620      3622  848300268X                     Dones (ColÂ¨lecciÃ³ clÃ ssica)   
3643      3645  0967231434       Small leather journal with strap-myra series 

In [ ]:
# Split the 'subject' column by the pipe (|) symbol
enriched_books_df['subjects'] = enriched_books_df['subjects'].str.split('|')

# Ensure any NaN values are replaced with an empty list so that they don't cause issues
enriched_books_df['subjects'] = enriched_books_df['subjects'].apply(lambda x: x if isinstance(x, list) else [])

# Flatten the lists in the 'subjects' column and count unique subjects
all_subjects = enriched_books_df['subjects'].explode().dropna()

# Count the unique subjects
unique_subject_count = all_subjects.nunique()

# Print the count of unique subjects
print(f"Number of unique subjects: {unique_subject_count}")

# Optional: If you want to print the actual unique subjects
unique_subjects = all_subjects.unique()
print(f"Unique subjects: {unique_subjects}")

# Create a list of unique subjects
df_subjects = unique_subjects

# Apply one-hot encoding for each unique subject
for subject in df_subjects:
    enriched_books_df[subject] = enriched_books_df['subjects'].apply(lambda x: 1 if subject in x else 0)

# Print the resulting DataFrame to check the changes
print(enriched_books_df.head())

Number of unique subjects: 3377
Unique subjects: ['History' 'Military' 'World War II' ... 'Rock Climbing'
 'Tibet (China) -- History -- 1951-'
 '19th Century American Novel And Short Story']


Streaming output truncated to the last 5000 lines.
<ipython-input-8-ba9328fd1ee8>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  enriched_books_df[subject] = enriched_books_df['subjects'].apply(lambda x: 1 if subject in x else 0)
<ipython-input-8-ba9328fd1ee8>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  enriched_books_df[subject] = enriched_books_df['subjects'].apply(lambda x: 1 if subject in x else 0)
<ipython-input-8-ba9328fd1ee8>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

   Book_ID      ISBN10                                         Book-Title  \
0        1  0060973129                               Decision in Normandy   
1        2  0393045218                             The Mummies of Urumchi   
2        3  0425176428  What If?: The World's Foremost Military Histor...   
3        4  0452264464               Beloved (Plume Contemporary Fiction)   
4        5  0609804618  Our Dumb Century: The Onion Presents 100 Years...   

       Book-Author  Year-Of-Publication                 Publisher  \
0     Carlo D'Este                 1991           HarperPerennial   
1  E. J. W. Barber                 1999    W. W. Norton & Company   
2    Robert Cowley                 2000  Berkley Publishing Group   
3    Toni Morrison                 1994                     Plume   
4        The Onion                 1999        Three Rivers Press   

          isbn13                                           subjects  \
0  9780060973124                  [History, Militar

<ipython-input-8-ba9328fd1ee8>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  enriched_books_df[subject] = enriched_books_df['subjects'].apply(lambda x: 1 if subject in x else 0)
<ipython-input-8-ba9328fd1ee8>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  enriched_books_df[subject] = enriched_books_df['subjects'].apply(lambda x: 1 if subject in x else 0)
<ipython-input-8-ba9328fd1ee8>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [ ]:
# List of ISBNs with missing subjects
missing_isbns = [
    "0894803700", "0676974562", "0884092097", "9510236756", "0553802550",
    "8448902378", "1410798860", "0679777547", "8475060099", "848300268X",
    "0967231434", "0597145806", "8090128920", "0613572742", "0868065951",
    "8806143581", "0743455967", "030768993X", "059033560X", "0909486727",
    "0441783295", "0312141890", "0536597073", "0449219089", "0736625828",
    "0891904891", "8486542480", "0962994235", "0140084096", "0130980056",
    "0749318627", "1929365438", "3635605360", "0860097374", "0973100117",
    "0451167724", "0880882239", "9035110617", "0425177378", "1413708986",
    "0670032549", "979428386X", "0671832107", "061398546X", "0850917565",
    "0340190175", "9503703689", "0671817086", "1842120646", "8821162028",
    "2890452107", "8474440904", "8429805931", "9072766997", "0835619060",
    "0590393871"
]

# Manually add subjects for each ISBN (for demonstration purposes, we're using a dictionary of ISBN -> subject)
isbn_to_subject = {
    "0894803700": ["Happiness"],
    "0676974562": ["Love", "War", "Psychological fiction", "Romance - Contemporary"],
    "0884092097": ["Poetry", "Coming of Age"],
    "9510236756": ["Biography / Autobiography", "Cultural Studies"],
    "0553802550": ["Parenting & Families", "Parenting"],
    "8448902378": ["Adventure", "Teen & Young Adult"],
    "1410798860": ["World War II", "Historical", "Memoirs", "Parenting Girls", "Parenting"],
    "0679777547": ["Mystery, Thriller & Suspense", "Adventure"],
    "8475060099": ["Contemporary", "Spain"],
    "848300268X": ["Women Authors"],
    "0967231434": ["Writing", "Journal Writing"],
    "0597145806": ["Business"],
    "8090128920": ["Coming of Age", "First Loves", "Teen & Young Adult"],
    "0613572742": ["Eastern", "Spiritual", "Religion & Spirituality"],
    "0868065951": ["Poetry"],
    "8806143581": ["Short Stories", "Satire"],
    "0743455967": ["Memoirs", "Writing"],
    "030768993X": ["Children's Books"],
    "059033560X": ["Family Relationships", "Divorce", "Behavioral Psychology", "Siblings"],
    "0909486727": ["Australia", "Australia & Oceania"],
    "0441783295": ["Fiction / Science Fiction / General", "Science Fiction"],
    "0312141890": ["Children's Books", "Coming of Age"],
    "0536597073": ["Africa", "Geography & Cultures"],
    "0449219089": ["Crime"],
    "0736625828": ["Romance - Historical", "Historical Fiction"],
    "0891904891": ["Greek & Roman", "Fantasy"],
    "8486542480": ["Short Stories"],
    "0962994235": ["Health", "Medicine", "Alternative Medicine"],
    "0140084096": ["Australia", "Holocaust"],
    "0130980056": ["Marketing & Sales"],
    "0749318627": ["Biography / Autobiography", "Love & Loss"],
    "1929365438": ["Schools & Teaching"],
    "3635605360": ["Erotica", "Sexuality", "Dreams"],
    "0860097374": ["Love"],
    "0973100117": ["Family Relationships"],
    "0451167724": ["Personal Transformation", "Time Management"],
    "0880882239": ["Self-Help", "Journal Writing"],
    "9035110617": ["History - Military / War", "History"],
    "0425177378": ["Contemporary", "Romance", "Family Relationships"],
    "1413708986": ["Mechanical", "Engineering"],
    "0670032549": ["Science Fiction & Fantasy", "Horror"],
    "979428386X": ["Economics"],
    "0671832107": ["Fiction", "Paranormal"],
    "061398546X": ["Fiction"],
    "0850917565": ["Children's Books"],
    "0340190175": ["Children's Books"],
    "9503703689": ["Political", "Memoirs"],
    "0671817086": ["Politics & Social Sciences"],
    "1842120646": ["Politics & Government"],
    "8821162028": ["Italy", "Fiction"],
    "2890452107": ["Travelers & Explorers"],
    "8474440904": ["Music", "Composers & Musicians"],
    "8429805931": ["Short Stories"],
    "9072766997": ["Cultural"],
    "0835619060": ["Alternative Medicine", "Meditation"],
    "0590393871": ["Children's Books"]
}

# Loop through each ISBN in the missing list
for isbn in missing_isbns:
    # Get the list of subjects for this ISBN (this could have multiple subjects)
    subjects = isbn_to_subject.get(isbn)

    if subjects:
        # For each subject, set the value to 1 in the corresponding column
        for subject in subjects:
            if subject in enriched_books_df.columns:
                enriched_books_df.loc[enriched_books_df['ISBN10'] == isbn, subject] = 1
            else:
                print(f"Subject '{subject}' does not exist as a column for ISBN: {isbn}")

# Flatten the list of subjects for use as column names
subject_columns = list(set([subject for subjects in isbn_to_subject.values() for subject in subjects]))

# Verify the update (print the updated rows)
print(enriched_books_df.loc[enriched_books_df['ISBN10'].isin(missing_isbns), ['ISBN10', 'subjects'] + subject_columns])



           ISBN10 subjects  Alternative Medicine  Romance - Historical  \
686    0894803700       []                     0                     0   
977    0676974562       []                     0                     0   
1145   0884092097       []                     0                     0   
1773   9510236756       []                     0                     0   
1815   0553802550       []                     0                     0   
1932   8448902378       []                     0                     0   
2715   1410798860       []                     0                     0   
3168   0679777547       []                     0                     0   
3501   8475060099       []                     0                     0   
3620   848300268X       []                     0                     0   
3643   0967231434       []                     0                     0   
4205   0597145806       []                     0                     0   
4270   8090128920       []            

In [ ]:
# Check for NaN values in each column
empty_columns = enriched_books_df.isna().sum()
# Print columns with missing values
print(empty_columns[empty_columns > 0])

synopsis    458
pages       868
dtype: int64


In [ ]:
# Save the DataFrame to a CSV file
enriched_books_df.to_csv('/content/enriched_books_dataset.csv', index=False)

In [ ]:
# Iterate over each subject column to count how many books have that subject (i.e., count the 1s)
subject_counts = enriched_books_df[df_subjects].sum()

# Sort the counts in descending order
sorted_subject_counts = subject_counts.sort_values(ascending=False)

# Print out each subject and its count, sorted from most to least common
for subject, count in sorted_subject_counts.items():
    print(f"Subject: {subject}, Count of books with this subject: {count}")

Subject: Literature & Fiction, Count of books with this subject: 9128
Subject: Genre Fiction, Count of books with this subject: 3294
Subject: Contemporary, Count of books with this subject: 3278
Subject: Mystery, Thriller & Suspense, Count of books with this subject: 3081
Subject: Romance, Count of books with this subject: 2640
Subject: United States, Count of books with this subject: 2410
Subject: Self Service, Count of books with this subject: 2003
Subject: Custom Stores, Count of books with this subject: 1983
Subject: Science Fiction & Fantasy, Count of books with this subject: 1980
Subject: Thrillers & Suspense, Count of books with this subject: 1955
Subject: Historical, Count of books with this subject: 1863
Subject: Mystery, Count of books with this subject: 1859
Subject: Literary, Count of books with this subject: 1681
Subject: Suspense, Count of books with this subject: 1302
Subject: Children's Books, Count of books with this subject: 1279
Subject: Fantasy, Count of books with 